### Compute and store filtered and demodulated velocity fields at each grid point

In [1]:
import numpy as np
import geopandas as gpd

import xarray as xr
from matplotlib import pyplot as plt
%matplotlib inline

from xhistogram.xarray import histogram
import dask.dataframe as dd
#import cartopy.crs as ccrs
#import cartopy.feature as cfeature

import mitequinox.utils as ut
from mitequinox.plot import *
import mitequinox.parcels as pa
from xmitgcm import llcreader

from scipy import signal
import scipy.ndimage as im

from sympy import Symbol, pi, atan, factor, lambdify

import mitequinox.plot as pl
import mitequinox.sigp as sp

from fsspec.implementations.local import LocalFileSystem

In [16]:
from dask.distributed import Client, LocalCluster
#
#cluster = LocalCluster()
#
from dask_jobqueue import PBSCluster
cluster = PBSCluster(processes=3, cores=3) #processes=7, cores=7
w = cluster.scale(jobs=15
                 )
client = Client(cluster)

In [14]:
client

<Client: No scheduler connected>

In [17]:
def convolve(x, h=None, hilbert=False):
    """ Convolve an input signal with a kernel
    Optionaly compute the Hilbert transform of the resulting time series
    
    Parameters
    x : input signal
    h : filter 
    hilbert : True for Hilbert transform to be applied to the filtered signal
    
    Returns
    x_f : filtered signal or hilbert transform of the filtered signal
    """
    x_f = signal.filtfilt(h, [1], x, axis=-1,padlen=0)#
    if hilbert:
        return signal.hilbert(x_f)
    else:
        return x_f

def filt(v, h, hilbert=False):
    
    output_dtype = complex if hilbert else float
    gufunc_kwargs = dict(output_sizes={'time': len(v.time)})
    return xr.apply_ufunc(convolve, v, kwargs={'h': h, 'hilbert': hilbert},
                    dask='parallelized', output_dtypes=[output_dtype],
                    input_core_dims=[['time']],
                    output_core_dims=[['time']],
                    dask_gufunc_kwargs = gufunc_kwargs,
                         )

##### First step : Filter and demodulate for each face

In [18]:
# Load Eulerian fields
ds = xr.open_zarr(ut.work_data_dir+'rechunked/SSU_rot.zarr',consolidated=False)
grd = ut.load_grd(consolidated=False)[['XC', 'YC', 'Depth']]#.persist()

In [19]:
#isel=dict(face=12) #face
#V = ['SSV_rot']#'zonal_velocity','meridional_velocity'
path = '/home1/datawork/zcaspar/mit4320/filtered_itide/'
dsel = 200

In [20]:
dt = 1/24 # time step in days

tidal_omega = sp.get_tidal_frequencies("M2", "K2","S2","N2")
omega_M2,omega_S2,omega_N2,omega_K2, domega, name = tidal_omega["M2"],tidal_omega["S2"],tidal_omega["N2"],tidal_omega["K2"], .2, "semidiurnal"
omega = (omega_M2+omega_S2)/2#center frequency
Tw = 30 #filter length
dband = 0.2 # half-bandwidth
V = ['SSU_rot']

In [21]:
#dt = 1. # in hours
#T = 20
#omega = 1/30
#h = signal.firwin(T*24, cutoff=[omega], pass_zero=True, nyq=1./2/dt, scale=True)
#V = 'SSU_rot'

In [22]:
import dask
def wrap_filter(_ds,dt=dt):
    h = sp.generate_filter(om, T=Tw, dt=dt, bandwidth=dband, normalized_bandwidth=None)
    time = np.arange(0,_ds['time'].values.size*dt,dt)
    exp = np.exp(-1j*om*2*np.pi*time)
#    _ds = _ds.chunk(dict(time=16,i=100,j=100))
    ds_hat={}
    for v in V:
        ds_hat[v+'_hat'] = filt(_ds[v], h,hilbert=False)#.persist()
#        ds_hat[v+'_hat'] = ds_hat[v+'_hat'].assign_coords({'lon':ds_hat[v+'_hat'].lon,'lat':ds_hat[v+'_hat'].lat})
        ds_hat[v+'_demodulated'] = ds_hat[v+'_hat']*exp
    ds_hat = xr.merge([ds_hat[v].rename(v) for v in ds_hat.keys()])
    ds_hat = ds_hat.chunk(dict(i=50)) #,j=4320//4
    ds_hat = ut._reset_chunk_encoding(ds_hat)
    return ds_hat#.drop(['XC','YC'])

def wrap_filter_low(_ds,dt=dt):
    h = signal.firwin(T*24, cutoff=[omega], pass_zero=True, nyq=1./2/dt, scale=True)
    ds_hat = filt(_ds[v], h,hilbert=False)#.persist()
    ds_hat = ds_hat.rename(v+'_hat').to_dataset()
#    ds_hat = ds_hat.chunk(dict(i=4320//4,j=4320//4))
#    ds_hat = ut._reset_chunk_encoding(ds_hat)
    return ds_hat

In [23]:
#Select and rechunk (empirical) original dataset
isel = dict(j=slice(0,None),face=slice(0,3))
dsp = ds.isel(**isel)#.isel(i=slice(4,None,4),j=slice(4, None,4))#.isel(face=0)#.sel(**sel)#.persist()
dsp = dsp.chunk({"time": -1})#.persist()#,'i':100,'j':70
dsp

<xarray.Dataset>
Dimensions:  (face: 3, j: 4320, i: 4320, time: 8640)
Coordinates:
  * face     (face) int64 0 1 2
  * i        (i) int64 0 1 2 3 4 5 6 7 ... 4313 4314 4315 4316 4317 4318 4319
  * j        (j) int64 0 1 2 3 4 5 6 7 ... 4313 4314 4315 4316 4317 4318 4319
  * time     (time) datetime64[ns] 2011-11-15 ... 2012-11-08T23:00:00
Data variables:
    CS       (face, j, i) float32 dask.array<chunksize=(1, 1080, 1080), meta=np.ndarray>
    Depth    (face, j, i) float32 dask.array<chunksize=(1, 1080, 1080), meta=np.ndarray>
    SN       (face, j, i) float32 dask.array<chunksize=(1, 1080, 1080), meta=np.ndarray>
    SSU_rot  (time, face, j, i) float32 dask.array<chunksize=(8640, 1, 192, 576), meta=np.ndarray>
    XC       (face, j, i) float32 dask.array<chunksize=(1, 1080, 1080), meta=np.ndarray>
    YC       (face, j, i) float32 dask.array<chunksize=(1, 1080, 1080), meta=np.ndarray>
    rA       (face, j, i) float32 dask.array<chunksize=(1, 1080, 1080), meta=np.ndarray>

In [24]:
om,Tw,dband,V = omega,Tw,dband,V #define parameters needed in wrap_filter

In [12]:
v = 'SSU_rot'
dsp = dsp.isel(face=0)

In [25]:
# Apply wrap_filter on the dataset and store the result for each face (seperated)
import os
#zarr_main = os.path.join(ut.root_data_dir, "filtered_itide/SSU_filtered_face4.zarr")
with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ds_out, zarr = ut.custom_distribute(dsp, 
                                        wrap_filter,
                                        overwrite=True,
                                        suffix="SSU_filtered_test.zarr",
                                        tmp_dir=os.path.join(ut.root_data_dir, "filtered_itide"),
                                        append=True,
#                                        face=1
                                        j=100
                                       )

0it [04:14, ?it/s]


KeyboardInterrupt: 

In [38]:
ds_out

,Array,Chunk
Bytes,2.35 TiB,26.37 MiB
Shape,"(4320, 4320, 8640)","(100, 1080, 16)"
Count,95041 Tasks,95040 Chunks
Type,complex128,numpy.ndarray
,Array,Chunk
Bytes,2.35 TiB,26.37 MiB
Shape,"(4320, 4320, 8640)","(100, 1080, 16)"
Count,95041 Tasks,95040 Chunks
Type,complex128,numpy.ndarray


In [15]:
#client.restart()
cluster.close()

##### Second step : Concat filtered and demodulated fields for each face

In [28]:
from dask.distributed import Client, LocalCluster
#
#cluster = LocalCluster()

#
from dask_jobqueue import PBSCluster
cluster = PBSCluster(processes=5, cores=5, walltime='04:00:00') #processes=7, cores=7 ,walltime='03:00:00'
w = cluster.scale(jobs=10
                 )
client = Client(cluster)

In [29]:
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: http://10.148.1.47:8787/status,
Dashboard: http://10.148.1.47:8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.148.1.47:33356,Workers: 0
Dashboard: http://10.148.1.47:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [26]:
ds0 = xr.open_zarr(ut.work_data_dir+'filtered_itide/SSU_filtered_%s.zarr'%0)
ds1 = xr.open_zarr(ut.work_data_dir+'filtered_itide/SSU_filtered_%s.zarr'%1)

In [31]:
ds0

,Array,Chunk
Bytes,1.91 TiB,911.25 MiB
Shape,"(13, 1080, 1080, 8640)","(1, 48, 144, 8640)"
Count,2393 Tasks,2392 Chunks
Type,complex128,numpy.ndarray
,Array,Chunk
Bytes,0.95 TiB,455.62 MiB
Shape,"(13, 1080, 1080, 8640)","(1, 48, 144, 8640)"
Count,2393 Tasks,2392 Chunks
Type,float64,numpy.ndarray


In [32]:
with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ds = xr.merge([ds0,ds1])

/home1/datahome/zcaspar/miniconda3/envs/croco/lib/python3.8/site-packages/xarray/core/indexing.py:1233: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  value = value[(slice(None),) * axis + (subkey,)]
/home1/datahome/zcaspar/miniconda3/envs/croco/lib/python3.8/site-packages/dask/array/core.py:4527: PerformanceWarning: Increasing number of chunks by factor of 13
  result = blockwise(
/home1/datahome/zcaspar/miniconda3/envs/croco/lib/python3.8/site-packages/xarray/core/indexing.py:1233: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.spli

KeyboardInterrupt: 

In [27]:
from dask.distributed import wait
path = '/home/datawork-lops-osi/equinox/mit4320/filtered_itide/'
ds = wait(xr.merge([xr.open_zarr(ut.work_data_dir+'filtered_itide/SSU_filtered_%s.zarr'%i) for i in np.arange(0,4)],compat='no_conflicts'))
#ds = xr.concat([xr.open_zarr(ut.work_data_dir+'filtered_itide/SSV_filtered_face%s.zarr'%i
#                           ) for i in np.arange(11,12)],dim='face')#.persist()
#ds = xr.concat([xr.open_zarr(ut.work_data_dir+'filtered_itide/SSV_filtered_%s.zarr'%i
#                           ) for i in np.arange(0,5)],dim='face')
#ds = xr.concat([xr.open_zarr(ut.work_data_dir+'filtered_itide/SSU_filtered_%s.zarr'%i
#                           ) for i in ['00','01']],dim='face')

/home1/datahome/zcaspar/miniconda3/envs/croco/lib/python3.8/site-packages/xarray/core/indexing.py:1233: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  value = value[(slice(None),) * axis + (subkey,)]
/home1/datahome/zcaspar/miniconda3/envs/croco/lib/python3.8/site-packages/dask/array/core.py:4527: PerformanceWarning: Increasing number of chunks by factor of 13
  result = blockwise(
/home1/datahome/zcaspar/miniconda3/envs/croco/lib/python3.8/site-packages/xarray/core/indexing.py:1233: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.spli

Traceback (most recent call last):
  File "/home1/datahome/zcaspar/miniconda3/envs/croco/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/dev/shm/pbs.8931328.datarmor0/ipykernel_7470/147414552.py", line 3, in <module>
    ds = wait(xr.merge([xr.open_zarr(ut.work_data_dir+'filtered_itide/SSU_filtered_%s.zarr'%i) for i in np.arange(0,4)],compat='no_conflicts'))
  File "/home1/datahome/zcaspar/miniconda3/envs/croco/lib/python3.8/site-packages/xarray/core/merge.py", line 905, in merge
    merge_result = merge_core(
  File "/home1/datahome/zcaspar/miniconda3/envs/croco/lib/python3.8/site-packages/xarray/core/merge.py", line 640, in merge_core
    variables, out_indexes = merge_collected(
  File "/home1/datahome/zcaspar/miniconda3/envs/croco/lib/python3.8/site-packages/xarray/core/merge.py", line 242, in merge_collected
    merged_vars[name] = unique_variable(name, variables, compat)
  File "/

TypeError: object of type 'NoneType' has no len()

In [26]:
l = [ds0]+[xr.open_zarr(ut.work_data_dir+'filtered_itide/SSU_filtered_%s.zarr'%i) for i in np.arange(0,4)]

In [24]:
l.append(xr.open_zarr(ut.work_data_dir+'filtered_itide/SSU_filtered_%s.zarr'%i) for i in np.arange(0,4))

In [27]:
l

[<xarray.Dataset>
 Dimensions:              (face: 13, j: 1080, i: 1080, time: 8640)
 Coordinates:
   * face                 (face) int64 0 1 2 3 4 5 6 7 8 9 10 11 12
   * i                    (i) int64 0 4 8 12 16 20 ... 4300 4304 4308 4312 4316
   * j                    (j) int64 0 4 8 12 16 20 ... 4300 4304 4308 4312 4316
   * time                 (time) datetime64[ns] 2011-11-15 ... 2012-11-08T23:0...
 Data variables:
     SSU_rot_demodulated  (face, j, i, time) complex128 dask.array<chunksize=(1, 48, 144, 8640), meta=np.ndarray>
     SSU_rot_hat          (face, j, i, time) float64 dask.array<chunksize=(1, 48, 144, 8640), meta=np.ndarray>,
 <xarray.Dataset>
 Dimensions:              (face: 13, j: 1080, i: 1080, time: 8640)
 Coordinates:
   * face                 (face) int64 0 1 2 3 4 5 6 7 8 9 10 11 12
   * i                    (i) int64 0 4 8 12 16 20 ... 4300 4304 4308 4312 4316
   * j                    (j) int64 0 4 8 12 16 20 ... 4300 4304 4308 4312 4316
   * time            

In [12]:
l = [ds0]+[xr.open_zarr(ut.work_data_dir+'filtered_itide/SSU_filtered_%s.zarr'%i) for i in np.arange(0,2)]
ds = xr.merge(l)

/home1/datahome/zcaspar/miniconda3/envs/croco/lib/python3.8/site-packages/xarray/core/indexing.py:1233: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  value = value[(slice(None),) * axis + (subkey,)]
/home1/datahome/zcaspar/miniconda3/envs/croco/lib/python3.8/site-packages/dask/array/core.py:4527: PerformanceWarning: Increasing number of chunks by factor of 13
  result = blockwise(
/home1/datahome/zcaspar/miniconda3/envs/croco/lib/python3.8/site-packages/xarray/core/indexing.py:1233: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.spli

KeyboardInterrupt: 

In [25]:
def concat(ds):
#    _ds = xr.concat([xr.open_zarr(ut.work_data_dir+'filtered_itide/SSV_filtered_%s.zarr'%i
#                           ) for i in np.arange(0,5)],dim='face')
#    _ds = ds#xr.concat(ds,dim='face')
#    l = [ds]
#    l.append(xr.open_zarr(ut.work_data_dir+'filtered_itide/SSU_filtered_%s.zarr'%i) for i in np.arange(1,4))
#    _ds = xr.merge([xr.open_zarr(ut.work_data_dir+'filtered_itide/SSU_filtered_%s.zarr'%i) for i in np.arange(0,4)],compat='no_conflicts')#
    _ds = ds.chunk(dict(time=16,i=4320//4,j=4320//4)) #,j=4320//4
    _ds = ut._reset_chunk_encoding(_ds)
    return _ds

In [6]:
ds = ds.persist()

In [33]:
#Apply concat function on ds
face = 1
import os, dask
with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ds_out, zarr = ut.custom_distribute(xr.merge([ds0,ds1]), 
                                        concat,
                                        overwrite=True,
                                        suffix="SSU_filtered.zarr",
                                        tmp_dir=os.path.join(ut.root_data_dir, "filtered_itide"),
                                        append=True,
                                        face=face
#                                        j=20
                                       )

/home1/datahome/zcaspar/miniconda3/envs/croco/lib/python3.8/site-packages/dask/array/core.py:4527: PerformanceWarning: Increasing number of chunks by factor of 13
  result = blockwise(
/home1/datahome/zcaspar/miniconda3/envs/croco/lib/python3.8/site-packages/dask/array/core.py:4527: PerformanceWarning: Increasing number of chunks by factor of 13
  result = blockwise(
/home1/datahome/zcaspar/miniconda3/envs/croco/lib/python3.8/site-packages/dask/array/core.py:4527: PerformanceWarning: Increasing number of chunks by factor of 13
  result = blockwise(
/home1/datahome/zcaspar/miniconda3/envs/croco/lib/python3.8/site-packages/dask/array/core.py:4527: PerformanceWarning: Increasing number of chunks by factor of 13
  result = blockwise(


KeyboardInterrupt: 

In [7]:
ds_out

,Array,Chunk
Bytes,7.04 TiB,18.46 MiB
Shape,"(3, 4320, 4320, 8640)","(1, 70, 1080, 16)"
Count,401761 Tasks,401760 Chunks
Type,complex128,numpy.ndarray
,Array,Chunk
Bytes,7.04 TiB,18.46 MiB
Shape,"(3, 4320, 4320, 8640)","(1, 70, 1080, 16)"
Count,401761 Tasks,401760 Chunks
Type,complex128,numpy.ndarray


In [28]:
#client.restart()
cluster.close()

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
ERROR:asyncio:_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
asyncio.exceptions.CancelledError
